In [1]:
from datasets import load_dataset, load_from_disk

from setfit import SetFitModel, SetFitTrainer

import torch
import numpy as np
import pandas as pd
# set to True to use the gpu (if there is one available)
use_gpu = True

# select device
if torch.cuda.is_available():
    device = torch.cuda.current_device()
else:
    device =  -1
print(device)

0


In [2]:
dataset = load_from_disk("data/belief_dataset/")

data_dict = dataset.train_test_split(test_size=0.95)
train_ds = data_dict['train']

for entry in train_ds:
    print(entry['label'])

2
2
0
0
0
2
0
1
2
0
2
0
2
2
0
3
0
2


In [3]:
test_ds = data_dict['test']

from sentence_transformers.losses import CosineSimilarityLoss

model = SetFitModel.from_pretrained("sentence-transformers/all-mpnet-base-v2")
trainer = SetFitTrainer(
    model=model,
    train_dataset=train_ds,
    eval_dataset=test_ds,
    loss_class=CosineSimilarityLoss,
    batch_size=4,
    num_iterations=20, # Number of text pairs to generate for contrastive learning
    num_epochs=1 # Number of epochs to use for contrastive learning
)

model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.


In [4]:
# Train and evaluate!
trainer.train()
metrics = trainer.evaluate()

***** Running training *****
  Num examples = 720
  Num epochs = 1
  Total optimization steps = 180
  Total train batch size = 4


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/180 [00:00<?, ?it/s]

***** Running evaluation *****


In [10]:
trainer.model._save_pretrained(save_directory="models/few-shot-mpnet-base-v2")

In [11]:
from transformers import pipeline
classifier = pipeline("zero-shot-classification",
                      model="models/few-shot-mpnet-base-v2/", device=device)

Some weights of the model checkpoint at models/few-shot-mpnet-base-v2/ were not used when initializing MPNetForSequenceClassification: ['pooler.dense.bias', 'pooler.dense.weight']
- This IS expected if you are initializing MPNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing MPNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of MPNetForSequenceClassification were not initialized from the model checkpoint at models/few-shot-mpnet-base-v2/ and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.weight', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task 

In [12]:
from transformers.pipelines.pt_utils import KeyDataset
from tqdm.auto import tqdm

labels = ["neither", "negative", "positive", "both"]
pred_i_list = []
for pred in tqdm(classifier(KeyDataset(test_ds, "text"), batch_size=2, candidate_labels=labels)):
    pred_i = labels.index(pred["labels"][0])
    pred_i_list.append(pred_i)

  0%|          | 0/173 [00:00<?, ?it/s]

c:\Users\conni\anaconda3\envs\tf-gpu\lib\site-packages\torch\utils\data\dataloader.py:529: UserWarning: Length of IterableDataset <transformers.pipelines.pt_utils.PipelineChunkIterator object at 0x000002992FF57FD0> was reported to be 345 (when accessing len(dataloader)), but 346 samples have been fetched. 
  warnings.warn(warn_msg)
c:\Users\conni\anaconda3\envs\tf-gpu\lib\site-packages\torch\utils\data\dataloader.py:529: UserWarning: Length of IterableDataset <transformers.pipelines.pt_utils.PipelineChunkIterator object at 0x000002992FF57FD0> was reported to be 345 (when accessing len(dataloader)), but 347 samples have been fetched. 
  warnings.warn(warn_msg)
c:\Users\conni\anaconda3\envs\tf-gpu\lib\site-packages\torch\utils\data\dataloader.py:529: UserWarning: Length of IterableDataset <transformers.pipelines.pt_utils.PipelineChunkIterator object at 0x000002992FF57FD0> was reported to be 345 (when accessing len(dataloader)), but 348 samples have been fetched. 
  warnings.warn(warn_msg

In [13]:
from sklearn.metrics import classification_report

y_true = test_ds["label"]
y_pred = pred_i_list
print(classification_report(y_true, y_pred, target_names=labels))

              precision    recall  f1-score   support

     neither       0.00      0.00      0.00       114
    negative       0.24      0.31      0.27        64
    positive       0.40      0.76      0.53       139
        both       0.00      0.00      0.00        28

    accuracy                           0.37       345
   macro avg       0.16      0.27      0.20       345
weighted avg       0.21      0.37      0.26       345



c:\Users\conni\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\conni\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\conni\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
